<a href="https://colab.research.google.com/github/beingmechon/DataAnalysisWithPython/blob/main/Electoral_Bond_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Old

In [ ]:
! pip install -q tabula-py squarify plotly dash pyspark jpype

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import tabula
import squarify
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# set the display format for float

pd.options.display.float_format = '{:.2f}'.format

In [ ]:
def pdf_to_df(path):
  pdf_output = tabula.read_pdf(path, pages='all', lattice=True)

  for i in range(1, len(pdf_output)):
    pdf_output[i].columns = pdf_output[0].columns

  return pd.concat(pdf_output, ignore_index=True)

In [ ]:
eb_parties = pdf_to_df("/content/EB-Parties.pdf")
eb_purchaser = pdf_to_df("/content/EB-Purchaser.pdf")


## Explore Purchaser data

In [ ]:
eb_purchaser.describe()

,Date of Purchase,Purchaser Name,Denomination
count,18535,18535,18535
unique,146,1317,5
top,05/Jan/2022,FUTURE GAMING AND HOTEL SERVICES PR,"1,00,00,000"
freq,559,1187,11472


In [ ]:
eb_purchaser.head()

,Date of Purchase,Purchaser Name,Denomination
0,12/Apr/2019,A B C INDIA LIMITED,"1,00,000"
1,12/Apr/2019,A B C INDIA LIMITED,"1,00,000"
2,12/Apr/2019,A B C INDIA LIMITED,"10,00,000"
3,12/Apr/2019,A B C INDIA LIMITED,"10,00,000"
4,12/Apr/2019,A B C INDIA LIMITED,"1,00,000"


In [ ]:
eb_purchaser.isnull().sum()

Date of Purchase    0
Purchaser Name      0
Denomination        0
dtype: int64

In [ ]:
eb_purchaser["Date of Purchase"] = pd.to_datetime(eb_purchaser["Date of Purchase"])
eb_purchaser['Denomination'] = eb_purchaser['Denomination'].str.replace(',', '').astype("float")
eb_purchaser.head()

,Date of Purchase,Purchaser Name,Denomination
0,2019-04-12,A B C INDIA LIMITED,100000.00
1,2019-04-12,A B C INDIA LIMITED,100000.00
2,2019-04-12,A B C INDIA LIMITED,1000000.00
3,2019-04-12,A B C INDIA LIMITED,1000000.00
4,2019-04-12,A B C INDIA LIMITED,100000.00


In [ ]:
# Take copy

eb_purchaser_copy = eb_purchaser.copy()

In [ ]:
import plotly.express as px

# Define n and display_top_n flag
n = 10
display_top_n = True

total_purchased = eb_purchaser.groupby("Purchaser Name")["Denomination"].sum()
total_purchased /= 1_00_00_000  # Convert to crores

total_purchased_sorted = total_purchased.sort_values(ascending=False)

if display_top_n:
    top_n_purchased = total_purchased_sorted.head(n)
    others_total = total_purchased_sorted.iloc[n:].sum()
    top_n_purchased['Others'] = others_total
    total_purchased_df = top_n_purchased.reset_index()
else:
    total_purchased_df = total_purchased_sorted.reset_index()

# Rename columns
total_purchased_df.columns = ['Purchaser Name', 'Total Purchased']

# Plot the treemap
fig = px.treemap(total_purchased_df,
                 path=['Purchaser Name'],
                 values='Total Purchased',
                 title='Total Purchased by Purchaser in Crores')

# Update layout and traces
fig.update_layout(margin=dict(t=50, l=50, r=50, b=50),
                  treemapcolorway=px.colors.qualitative.Plotly)
fig.update_traces(marker=dict(cornerradius=5),
                  textinfo='label+value',
                  hoverinfo='label+value+percent entry',
                  textfont=dict(color='white', size=12))

fig.show()


In [ ]:
n = 50

fig2 = px.bar(total_purchased_sorted.head(n).sort_values(ascending=True),
              color='value',
              title=f'Top {n} Bonds Purchasers (Amount in Crores)',
              text_auto='.4s',
              orientation='h')
fig2.update_traces(textfont_size=12, textangle=0, cliponaxis=False, textposition='outside')
fig2.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', height=800)
fig2.show()

In [ ]:
df = eb_purchaser.groupby(['Date of Purchase', 'Purchaser Name'])['Denomination'].sum().unstack().reset_index().fillna(0)
df.head()

Purchaser Name,Date of Purchase,14 REELS PLUS LLP,A B C INDIA LIMITED,AAKANKSHA BAHETY,AALAYA CONSTRUCTIONS,AARISH SOLAR POWER PRIVATE LIMITED,AASHMAN ENERGY PRIVATE LIMITED,AASHMAN ENERGY PVT LTD,AAYTEE LOGISTICS PRIVATE LIMITED,ABHAY SHUKLA,...,YASHODA SUPER SPECIALITY HOSPITAL,YOGA BUILDERS PVT LTD,YOGESH GARODIA,YOGESH MANNALAL AGRAWAL,YOGESH MANPURIA,YUVAN TRADINGCONSULTANCY LLP,ZENSAR TECHNOLOGIES LTD,ZUVAN ENERGY PRIVATE LIMITED,ZUVAN ENERGY PVT LTD,ZYDUS HEALTHCARE LIMITED
0,2019-04-12,0.00,4000000.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2019-04-15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2019-04-16,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2019-04-17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2019-04-18,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

df = eb_purchaser.groupby(['Date of Purchase', 'Purchaser Name'])['Denomination'].sum().unstack().reset_index().fillna(0)

app = Dash(__name__)

purchaser_names = eb_purchaser['Purchaser Name'].unique()
options = [{'label': name, 'value': name} for name in purchaser_names]

app.layout = html.Div([
    html.H4('Electoral Bond Purchaser Analysis'),
    dcc.Graph(id="PurchaserPlot"),
    html.P("Select Purchaser:"),
    dcc.Dropdown(
        id="ticker",
        options=options,
        value=purchaser_names[0],
        clearable=False,
    ),
])

@app.callback(
    Output("PurchaserPlot", "figure"),
    Input("ticker", "value"))
def display_time_series(ticker):
    fig = px.area(df, x='Date of Purchase', y=ticker)
    return fig

app.run_server(debug=True)


<IPython.core.display.Javascript object>

## Explore Party Data

In [ ]:
eb_parties.head()

,Date of\rEncashment,Name of the Political Party,Denomination
0,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,"10,00,000"
1,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,"10,00,000"
2,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,"1,00,00,000"
3,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,"10,00,000"
4,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,"10,00,000"


In [ ]:
eb_parties.describe()

,Date of\rEncashment,Name of the Political Party,Denomination
count,19996,19996,19996
unique,230,27,5
top,18/Apr/2019,BHARTIYA JANTA PARTY,"1,00,00,000"
freq,584,8444,11958


In [ ]:
eb_parties.isnull().sum()

Date of\rEncashment            0
Name of the Political Party    0
Denomination                   0
dtype: int64

In [ ]:
eb_parties['Denomination'].unique()

array(['10,00,000', '1,00,00,000', '1,00,000', '10,000', '1,000'],
      dtype=object)

In [ ]:
eb_parties["Date of\rEncashment"] = pd.to_datetime(eb_parties["Date of\rEncashment"])
eb_parties['Denomination'] = eb_parties['Denomination'].astype(str).str.replace(',', '').astype("float")
eb_parties.head()

,Date of\rEncashment,Name of the Political Party,Denomination
0,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.00
1,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.00
2,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,10000000.00
3,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.00
4,2019-04-12,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,1000000.00


In [ ]:
# Take copy

eb_parties_copy = eb_parties.copy()

In [ ]:
eb_parties['Denomination'].unique()

array([1.e+06, 1.e+07, 1.e+05, 1.e+04, 1.e+03])

In [ ]:
import plotly.express as px

# Define n and display_top_n flag
n = 10
display_top_n = True

total_enchased = eb_parties.groupby("Name of the Political Party")["Denomination"].sum()
total_enchased /= 1_00_00_000  # Convert to crores

total_enchased_sorted = total_enchased.sort_values(ascending=False)

if display_top_n:
    top_n_enchased = total_enchased_sorted.head(n)
    others_total = total_enchased_sorted.iloc[n:].sum()
    top_n_enchased['Others'] = others_total
    total_enchased_df = top_n_enchased.reset_index()
else:
    total_enchased_df = total_enchased_sorted.reset_index()

# Rename columns
total_enchased_df.columns = ["Name of the Political Party", 'Total Enchased']

# Plot the treemap
fig = px.treemap(total_enchased_df,
                 path=['Name of the Political Party'],
                 values='Total Enchased',
                 title='Total Enchased by Party in Crores')

# Update layout and traces
fig.update_layout(margin=dict(t=50, l=50, r=50, b=50),
                  treemapcolorway=px.colors.qualitative.Plotly)
fig.update_traces(marker=dict(cornerradius=5),
                  textinfo='label+value',
                  hoverinfo='label+value+percent entry',
                  textfont=dict(color='white', size=12))

fig.show()


In [ ]:
n = 50

fig2 = px.bar(total_enchased_sorted.head(n).sort_values(ascending=True),
              color='value',
              title=f'Top {n} Bonds Enchased Parties (Amount in Crores)',
              text_auto='.4s',
              orientation='h')
fig2.update_traces(textfont_size=12, textangle=0, cliponaxis=False, textposition='outside')
fig2.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', height=800)
fig2.show()

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

df = eb_parties.groupby(['Date of\rEncashment', "Name of the Political Party"])['Denomination'].sum().unstack().reset_index().fillna(0)

app = Dash(__name__)

party_names = eb_parties['Name of the Political Party'].unique()
options = [{'label': name, 'value': name} for name in party_names]

app.layout = html.Div([
    html.H4('Electoral Bond Enchased Party Details'),
    dcc.Graph(id="EnchasedPlot"),
    html.P("Select Enchaser:"),
    dcc.Dropdown(
        id="ticker",
        options=options,
        value=party_names[0],
        clearable=False,
    ),
])

@app.callback(
    Output("EnchasedPlot", "figure"),
    Input("ticker", "value"))
def display_time_series(ticker):
    fig = px.area(df, x='Date of\rEncashment', y=ticker)
    return fig

app.run_server(debug=True)


<IPython.core.display.Javascript object>

## Analysis using PySpark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Electoral Bond Analysis").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").getOrCreate()

In [ ]:
eb_purchaser_spark = spark.createDataFrame(eb_purchaser_copy)
eb_parties_spark = spark.createDataFrame(eb_parties_copy)

In [ ]:
eb_purchaser_spark = eb_purchaser_spark.withColumnRenamed("Purchaser Name", "Name")\
                                        .withColumnRenamed("Date of Purchase", "Date")

eb_purchaser_spark.show(5, 0), eb_purchaser_spark.dtypes

+-------------------+-------------------+------------+
|Date               |Name               |Denomination|
+-------------------+-------------------+------------+
|2019-04-12 00:00:00|A B C INDIA LIMITED|100000.0    |
|2019-04-12 00:00:00|A B C INDIA LIMITED|100000.0    |
|2019-04-12 00:00:00|A B C INDIA LIMITED|1000000.0   |
|2019-04-12 00:00:00|A B C INDIA LIMITED|1000000.0   |
|2019-04-12 00:00:00|A B C INDIA LIMITED|100000.0    |
+-------------------+-------------------+------------+
only showing top 5 rows



(None, [('Date', 'timestamp'), ('Name', 'string'), ('Denomination', 'double')])

In [ ]:
eb_parties_spark = eb_parties_spark.withColumnRenamed("Name of the Political Party", "Name")\
                                    .withColumnRenamed("Date of\rEncashment", "Date")

eb_parties_spark.show(5, 0), eb_parties_spark.dtypes

+-------------------+----------------------------------------+------------+
|Date               |Name                                    |Denomination|
+-------------------+----------------------------------------+------------+
|2019-04-12 00:00:00|ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM|1000000.0   |
|2019-04-12 00:00:00|ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM|1000000.0   |
|2019-04-12 00:00:00|ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM|1.0E7       |
|2019-04-12 00:00:00|ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM|1000000.0   |
|2019-04-12 00:00:00|ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM|1000000.0   |
+-------------------+----------------------------------------+------------+
only showing top 5 rows



(None, [('Date', 'timestamp'), ('Name', 'string'), ('Denomination', 'double')])

In [ ]:
eb_parties_spark.count(), eb_purchaser_spark.count()

(19996, 18535)

## Party and Purchase distinct count

Let see how many unique parties and purchasers present in the data

In [ ]:
print("Total unique count of Purchasers: ", eb_purchaser_spark.select('Name').distinct().count())
print("Total unique count of Parties: ", eb_parties_spark.select('Name').distinct().count())


Total unique count of Purchasers:  1317
Total unique count of Parties:  27


## Top funds Enchasers and Purchasers

In [ ]:
# eb_purchaser_spark.select('Denomination').show()
eb_purchaser_spark.dtypes

[('Date', 'timestamp'), ('Name', 'string'), ('Denomination', 'double')]

In [ ]:
eb_purchaser_spark = eb_purchaser_spark.withColumn("Denomination", eb_purchaser_spark.Denomination.cast('float'))

In [ ]:
topPurchasers = eb_purchaser_spark.groupBy('Name')\
                  .agg(sum('Denomination').alias('Total_Purchased_Amount')).orderBy(desc("Total_Purchased_Amount"))

topPurchasers.show()

+--------------------+----------------------+
|                Name|Total_Purchased_Amount|
+--------------------+----------------------+
|FUTURE GAMING AND...|              1.187E10|
|MEGHA ENGINEERING...|                8.08E9|
|QWIKSUPPLYCHAINPR...|                4.02E9|
|HALDIA ENERGY LIM...|                3.69E9|
|     VEDANTA LIMITED|              3.6864E9|
|ESSEL MINING AND ...|               2.214E9|
|WESTERN UP POWER ...|                2.17E9|
|KEVENTER FOODPARK...|                1.92E9|
|       MADANLAL LTD.|               1.815E9|
|BHARTI AIRTEL LIM...|                1.79E9|
|YASHODA SUPER SPE...|                1.58E9|
|UTKAL ALUMINA INT...|               1.343E9|
|MKJ ENTERPRISES L...|              1.2714E9|
|DLF COMMERCIAL DE...|                1.27E9|
|JINDAL STEEL AND ...|                1.21E9|
|B G SHIRKE CONSTR...|                1.14E9|
|DHARIWAL INFRASTR...|                1.13E9|
|CHENNAI GREEN WOO...|                1.04E9|
|BIRLACARBONINDIAP...|            

In [ ]:
topEnchasers = eb_parties_spark.groupBy('Name')\
                  .agg(sum('Denomination').alias('Total_Enchased_Amount')).orderBy(desc("Total_Enchased_Amount"))

topEnchasers.show(truncate=False)

+----------------------------------------+---------------------+
|Name                                    |Total_Enchased_Amount|
+----------------------------------------+---------------------+
|BHARTIYA JANTA PARTY                    |5.9288681E10         |
|ALL INDIA TRINAMOOL CONGRESS            |1.5743304E10         |
|PRESIDENT, ALL INDIA CONGRESS COMMITTEE |1.3961035E10         |
|BHARAT RASHTRA SAMITHI                  |1.1930389E10         |
|BIJU JANATA DAL                         |7.602E9              |
|DMK PARTY IN PARLIAMENT                 |6.24E9               |
|YSR CONGRESS PARTY (YUVAJANA SRAMIKA R  |3.2879E9             |
|TELUGU DESAM PARTY                      |2.1478E9             |
|SHIVSENA (POLITICAL PARTY)              |1.552613E9           |
|RASTRIYA JANTA DAL                      |7.21E8               |
|AAM AADMI PARTY                         |6.533E8              |
|JANATA DAL ( SECULAR )                  |4.34E8               |
|SIKKIM KRANTIKARI MORCHA

## Count of fund received by parties and purchased by purchasers

Lets see total count of fund recevied by party and bond purchased by purchaser

In [ ]:
eb_purchaser_spark.groupBy('Name').agg(count('Denomination').alias("Total_Count")).orderBy(desc("Total_Count")).show(10)

+--------------------+-----------+
|                Name|Total_Count|
+--------------------+-----------+
|FUTURE GAMING AND...|       1187|
|MEGHA ENGINEERING...|        808|
|QWIKSUPPLYCHAINPR...|        402|
|HALDIA ENERGY LIM...|        387|
|     VEDANTA LIMITED|        378|
|ESSEL MINING AND ...|        234|
|MKJ ENTERPRISES L...|        231|
|TRANSWAYS EXIM PR...|        223|
|WESTERN UP POWER ...|        217|
|KEVENTER FOODPARK...|        201|
+--------------------+-----------+
only showing top 10 rows



In [ ]:
eb_parties_spark.groupBy('Name').agg(count('Denomination').alias("Total_Count")).orderBy(desc("Total_Count")).show(10)

+--------------------+-----------+
|                Name|Total_Count|
+--------------------+-----------+
|BHARTIYA JANTA PARTY|       8444|
|ALL INDIA TRINAMO...|       3231|
|PRESIDENT, ALL IN...|       3088|
|BHARAT RASHTRA SA...|       1771|
|     BIJU JANATA DAL|        843|
|DMK PARTY IN PARL...|        633|
|YSR CONGRESS PART...|        461|
|SHIVSENA (POLITIC...|        347|
|  TELUGU DESAM PARTY|        274|
|     AAM AADMI PARTY|        242|
+--------------------+-----------+
only showing top 10 rows



## Party and

In [ ]:
party_list = [row.Name for row in eb_parties_spark.select("Name").distinct().collect()]

party_list

['BIHAR PRADESH JANTA DAL(UNITED)',
 'NATIONALIST CONGRESS PARTY PARLIAMENT OF',
 'NATIONALIST CONGRESS PARTY MAHARASHTRA PRADESH',
 'BHARTIYA JANTA PARTY',
 'SHIROMANI AKALI DAL',
 'ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM',
 'JAMMU AND KASHMIR NATIONAL CONFERENCE',
 'AAM AADMI PARTY',
 'BIJU JANATA DAL',
 'BHARAT RASHTRA SAMITHI',
 'GOA FORWARD PARTY',
 'JHARKHAND MUKTI MORCHA',
 'RASTRIYA JANTA DAL',
 'DMK PARTY IN PARLIAMENT',
 'YSR CONGRESS PARTY (YUVAJANA SRAMIKA R',
 'SHIVSENA',
 'SHIVSENA (POLITICAL PARTY)',
 'ALL INDIA TRINAMOOL CONGRESS',
 'JANATA DAL ( SECULAR )',
 'TELUGU DESAM PARTY',
 'ADYAKSHA SAMAJVADI PARTY',
 'MAHARASHTRAWADI GOMNTAK PARTY',
 'PRESIDENT, ALL INDIA CONGRESS COMMITTEE',
 'SIKKIM DEMOCRATIC FRONT',
 'SIKKIM KRANTIKARI MORCHA',
 'JANASENA PARTY',
 'RASHTRIYA JANTA DAL']

# New

In [1]:
! pip install -q tabula-py squarify plotly dash pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import tabula
import squarify
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from dash import Dash, dcc, html, Input, Output


In [10]:
def pdf_to_df(path):
  pdf_output = tabula.read_pdf(path, pages='all', lattice=True)

  for i in range(1, len(pdf_output)):
    pdf_output[i].columns = pdf_output[0].columns

  return pd.concat(pdf_output, ignore_index=True)

In [11]:
# !wget -O ECI-Enchasers.pdf https://raw.githubusercontent.com/beingmechon/DataAnalysisWithPython/main/electoral_bond/ECI-Enchasers.pdf
# !wget -O ECI-Purchasers.pdf https://raw.githubusercontent.com/beingmechon/DataAnalysisWithPython/main/electoral_bond/ECI-Purchasers.pdf

In [12]:
enchaser_pdf = "https://raw.githubusercontent.com/beingmechon/DataAnalysisWithPython/main/electoral_bond/ECI-Enchasers.pdf"
purchaser_pdf = "https://raw.githubusercontent.com/beingmechon/DataAnalysisWithPython/main/electoral_bond/ECI-Purchasers.pdf"

In [13]:
enchasers = pdf_to_df(enchaser_pdf)
purchasers = pdf_to_df(purchaser_pdf)

## Explore and Wrangle the data

### Explore dataframes

In [14]:
enchasers.head()

,Sr No.,Date of\rEncashment,Name of the Political Party,Account no. of\rPolitical Party,Prefix,Bond\rNumber,Denominations,Pay Branch\rCode,Pay Teller
0,1,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,OC,775,"1,00,00,000",800,2770121
1,2,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,OC,3975,"1,00,00,000",800,2770121
2,3,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,OC,3967,"1,00,00,000",800,2770121
3,4,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,TL,10418,"10,00,000",800,2770121
4,5,12/Apr/2019,ALL INDIA ANNA DRAVIDA MUNNETRA KAZHAGAM,*******5199,TL,126,"10,00,000",800,2770121


In [15]:
purchasers.head()

,Sr No.,Reference No (URN),Journal Date,Date of\rPurchase,Date of Expiry,Name of the Purchaser,Prefix,Bond\rNumber,Denominations,Issue Branch Code,Issue Teller,Status
0,1,00001201904120000001166,12/Apr/2019,12/Apr/2019,26/Apr/2019,A B C INDIA LIMITED,TL,11448,"10,00,000",1,5899230,Paid
1,2,00001201904120000001166,12/Apr/2019,12/Apr/2019,26/Apr/2019,A B C INDIA LIMITED,TL,11447,"10,00,000",1,5899230,Paid
2,3,00001201904120000001166,12/Apr/2019,12/Apr/2019,26/Apr/2019,A B C INDIA LIMITED,TL,11441,"10,00,000",1,5899230,Paid
3,4,00001201904120000001166,12/Apr/2019,12/Apr/2019,26/Apr/2019,A B C INDIA LIMITED,OL,1113,"1,00,000",1,5899230,Paid
4,5,00001201904120000001166,12/Apr/2019,12/Apr/2019,26/Apr/2019,A B C INDIA LIMITED,OL,1118,"1,00,000",1,5899230,Paid


In [16]:
purchasers.describe()

,Sr No.,Bond\rNumber,Issue Branch Code,Issue Teller
count,18871.000000,18871.000000,18871.000000,1.887100e+04
mean,9436.000000,12181.003338,480.065550,5.895737e+06
std,5447.732801,6573.754865,395.446834,1.825395e+06
min,1.000000,8.000000,1.000000,1.013030e+06
25%,4718.500000,8317.500000,1.000000,5.054982e+06
50%,9436.000000,12350.000000,509.000000,6.405134e+06
75%,14153.500000,14764.000000,813.000000,7.273126e+06
max,18871.000000,71548.000000,1355.000000,8.492239e+06


In [17]:
purchasers.isnull().sum()

Sr No.                   0
Reference No  (URN)      0
Journal Date             0
Date of\rPurchase        0
Date of Expiry           0
Name of the Purchaser    0
Prefix                   0
Bond\rNumber             0
Denominations            0
Issue Branch Code        0
Issue Teller             0
Status                   0
dtype: int64

In [18]:
enchasers.describe()

,Sr No.,Bond\rNumber,Pay Branch\rCode,Pay Teller
count,20421.000000,20421.000000,20421.000000,2.042100e+04
mean,10211.000000,11783.435140,559.467852,5.293055e+06
std,5895.179259,6525.685559,297.836567,1.953104e+06
min,1.000000,10.000000,1.000000,1.498450e+05
25%,5106.000000,7797.000000,691.000000,3.490777e+06
50%,10211.000000,11913.000000,691.000000,5.614473e+06
75%,15316.000000,14585.000000,691.000000,7.516991e+06
max,20421.000000,71548.000000,2295.000000,8.212066e+06


In [19]:
enchasers.isnull().sum()

Sr No.                             0
Date of\rEncashment                0
Name of the Political Party        0
Account no. of\rPolitical Party    0
Prefix                             0
Bond\rNumber                       0
Denominations                      0
Pay Branch\rCode                   0
Pay Teller                         0
dtype: int64

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Electoral Bond Analysis").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").getOrCreate()


In [21]:
enchaser_df = spark.createDataFrame(enchasers)
purchaser_df = spark.createDataFrame(purchasers)


In [22]:
enchaser_df_c = enchaser_df.alias('enchaser_df_c')
purchaser_df_c = purchaser_df.alias('purchaser_df_c')

### Remove unwnated Columns

In [23]:
# Remove unwanted cols from purchasers and enchasers

enchaser_df = enchaser_df.drop('Account no. of\rPolitical Party', 'Pay Teller', 'Pay Branch\rCode')
purchaser_df = purchaser_df.drop('Reference No  (URN)', 'Journal Date', 'Issue Branch Code', 'Issue Teller')

### Rename Columns

In [24]:
enchaser_df = enchaser_df.withColumnsRenamed({'Sr No.': 'SrNo',
                                              'Date of\rEncashment': 'EnchasedDate',
                                              'Name of the Political Party': 'EnchaserName',
                                              'Bond\rNumber': 'BondNumber'})

In [25]:
purchaser_df = purchaser_df.withColumnsRenamed({'Sr No.': 'SrNo',
                                                'Date of\rPurchase': 'PurchasedDate',
                                                'Date of Expiry': 'ExpiryDate',
                                                'Name of the Purchaser': 'PurchaserName',
                                                'Bond\rNumber': 'BondNumber'})

### Make unique identifier
Merged prefix and existing bond numbers to make new Bond Number



In [26]:
# Merged prefix and bond_number

enchaser_df = enchaser_df.withColumn('BondNo', concat(col('Prefix'), lit('-'), col('BondNumber')))
purchaser_df = purchaser_df.withColumn('BondNo', concat(col('Prefix'), lit('-'), col('BondNumber')))


In [27]:
# Removed merged columns

enchaser_df = enchaser_df.drop('Prefix', 'BondNumber')
purchaser_df = purchaser_df.drop('Prefix', 'BondNumber')

In [28]:
enchaser_df.show(2)

+----+------------+--------------------+-------------+-------+
|SrNo|EnchasedDate|        EnchaserName|Denominations| BondNo|
+----+------------+--------------------+-------------+-------+
|   1| 12/Apr/2019|ALL INDIA ANNA DR...|  1,00,00,000| OC-775|
|   2| 12/Apr/2019|ALL INDIA ANNA DR...|  1,00,00,000|OC-3975|
+----+------------+--------------------+-------------+-------+
only showing top 2 rows



### Split date

In [29]:
enchaser_df = enchaser_df.withColumn("EnchasedDate", to_date(enchaser_df["EnchasedDate"], "dd/MMM/yyyy"))
enchaser_df = enchaser_df.withColumn("Day", dayofmonth("EnchasedDate")) \
                         .withColumn("Month", month("EnchasedDate")) \
                         .withColumn("Year", year("EnchasedDate"))

enchaser_df.show(5)

+----+------------+--------------------+-------------+--------+---+-----+----+
|SrNo|EnchasedDate|        EnchaserName|Denominations|  BondNo|Day|Month|Year|
+----+------------+--------------------+-------------+--------+---+-----+----+
|   1|  2019-04-12|ALL INDIA ANNA DR...|  1,00,00,000|  OC-775| 12|    4|2019|
|   2|  2019-04-12|ALL INDIA ANNA DR...|  1,00,00,000| OC-3975| 12|    4|2019|
|   3|  2019-04-12|ALL INDIA ANNA DR...|  1,00,00,000| OC-3967| 12|    4|2019|
|   4|  2019-04-12|ALL INDIA ANNA DR...|    10,00,000|TL-10418| 12|    4|2019|
|   5|  2019-04-12|ALL INDIA ANNA DR...|    10,00,000|  TL-126| 12|    4|2019|
+----+------------+--------------------+-------------+--------+---+-----+----+
only showing top 5 rows



In [30]:
purchaser_df = purchaser_df.withColumn("PurchasedDate", to_date(purchaser_df["PurchasedDate"], "dd/MMM/yyyy"))
purchaser_df = purchaser_df.withColumn("Day", dayofmonth("PurchasedDate")) \
                         .withColumn("Month", month("PurchasedDate")) \
                         .withColumn("Year", year("PurchasedDate"))

purchaser_df.show(5)

+----+-------------+-----------+-------------------+-------------+------+--------+---+-----+----+
|SrNo|PurchasedDate| ExpiryDate|      PurchaserName|Denominations|Status|  BondNo|Day|Month|Year|
+----+-------------+-----------+-------------------+-------------+------+--------+---+-----+----+
|   1|   2019-04-12|26/Apr/2019|A B C INDIA LIMITED|    10,00,000|  Paid|TL-11448| 12|    4|2019|
|   2|   2019-04-12|26/Apr/2019|A B C INDIA LIMITED|    10,00,000|  Paid|TL-11447| 12|    4|2019|
|   3|   2019-04-12|26/Apr/2019|A B C INDIA LIMITED|    10,00,000|  Paid|TL-11441| 12|    4|2019|
|   4|   2019-04-12|26/Apr/2019|A B C INDIA LIMITED|     1,00,000|  Paid| OL-1113| 12|    4|2019|
|   5|   2019-04-12|26/Apr/2019|A B C INDIA LIMITED|     1,00,000|  Paid| OL-1118| 12|    4|2019|
+----+-------------+-----------+-------------------+-------------+------+--------+---+-----+----+
only showing top 5 rows



### Change column data types

In [31]:
enchaser_df = enchaser_df.withColumn("Denominations", col("Denominations").cast("string")) \
    .withColumn("Denominations", regexp_replace(col("Denominations"), ",", "")) \
    .withColumn("Denominations", col("Denominations").cast("float"))

In [32]:
purchaser_df = purchaser_df.withColumn("Denominations", col("Denominations").cast("string")) \
    .withColumn("Denominations", regexp_replace(col("Denominations"), ",", "")) \
    .withColumn("Denominations", col("Denominations").cast("float"))

## Analysis on Enchasers

In [33]:
from pyspark.sql.functions import sum as _sum

In [34]:
enchaser_df.show(5)

+----+------------+--------------------+-------------+--------+---+-----+----+
|SrNo|EnchasedDate|        EnchaserName|Denominations|  BondNo|Day|Month|Year|
+----+------------+--------------------+-------------+--------+---+-----+----+
|   1|  2019-04-12|ALL INDIA ANNA DR...|        1.0E7|  OC-775| 12|    4|2019|
|   2|  2019-04-12|ALL INDIA ANNA DR...|        1.0E7| OC-3975| 12|    4|2019|
|   3|  2019-04-12|ALL INDIA ANNA DR...|        1.0E7| OC-3967| 12|    4|2019|
|   4|  2019-04-12|ALL INDIA ANNA DR...|    1000000.0|TL-10418| 12|    4|2019|
|   5|  2019-04-12|ALL INDIA ANNA DR...|    1000000.0|  TL-126| 12|    4|2019|
+----+------------+--------------------+-------------+--------+---+-----+----+
only showing top 5 rows



In [35]:
topEnchaser = enchaser_df.groupBy('EnchaserName').agg(_sum('Denominations').alias('TotalAmount')).orderBy('TotalAmount', ascending=False)
topEnchaser.show(5, truncate=False)

+---------------------------------------+------------+
|EnchaserName                           |TotalAmount |
+---------------------------------------+------------+
|BHARATIYA JANATA PARTY                 |6.0605111E10|
|ALL INDIA TRINAMOOL CONGRESS           |1.6095314E10|
|PRESIDENT, ALL INDIA CONGRESS COMMITTEE|1.4218655E10|
|BHARAT RASHTRA SAMITHI                 |1.2147099E10|
|BIJU JANATA DAL                        |7.755E9     |
+---------------------------------------+------------+
only showing top 5 rows



In [37]:
n = 50

top_n_parties = topEnchaser.orderBy('TotalAmount', descending=False).limit(n)

fig1 = px.bar(top_n_parties.toPandas(),
              y='EnchaserName',
              x='TotalAmount',
              color='TotalAmount',
              title=f'Top {n} Bonds Enchased Parties (Amount in Crores)',
              text_auto='.2s',
              orientation='h')

fig1.update_traces(textfont_size=12, textangle=0, cliponaxis=False, textposition='outside')
fig1.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', height=800)
fig1.show()

In [38]:
totalByYear = enchaser_df.groupBy('EnchaserName', 'Year').agg(_sum('Denominations').alias('TotalAmountByYear')).orderBy('EnchaserName', 'Year', 'TotalAmountByYear')
totalByYear.show(5, truncate=False)

+---------------+----+-----------------+
|EnchaserName   |Year|TotalAmountByYear|
+---------------+----+-----------------+
|AAM AADMI PARTY|2019|2.2E7            |
|AAM AADMI PARTY|2020|3.3E7            |
|AAM AADMI PARTY|2021|1.85E7           |
|AAM AADMI PARTY|2022|4.795E8          |
|AAM AADMI PARTY|2023|1.015E8          |
+---------------+----+-----------------+
only showing top 5 rows



In [39]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

totalByYearPD = totalByYear.toPandas()
partyName = [row.EnchaserName for row in totalByYear.select('EnchaserName').distinct().collect()]


app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='Party Enchased per Year', style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='dropdown-selection',
        options=[{'label': party, 'value': party} for party in totalByYearPD['EnchaserName'].unique()],
        value='AAM AADMI PARTY'
    ),
    dcc.Graph(id='graph-content')
])

@app.callback(
    Output('graph-content', 'figure'),
    [Input('dropdown-selection', 'value')]
)
def update_graph(value):
    df = totalByYearPD[totalByYearPD['EnchaserName'] == value]
    fig = px.bar(df, x='Year', y='TotalAmountByYear', color='TotalAmountByYear')
    fig.update_xaxes(type='category')
    fig.update_layout(bargap=0.3)

    return fig

if __name__ == '__main__':
     app.run_server(mode='inline', host="0.0.0.0", port=1005)


<IPython.core.display.Javascript object>

## Analysis on Purchasers

In [40]:
from pyspark.sql.functions import sum as _sum

In [41]:
purchaser_df.show(5)

+----+-------------+-----------+-------------------+-------------+------+--------+---+-----+----+
|SrNo|PurchasedDate| ExpiryDate|      PurchaserName|Denominations|Status|  BondNo|Day|Month|Year|
+----+-------------+-----------+-------------------+-------------+------+--------+---+-----+----+
|   1|   2019-04-12|26/Apr/2019|A B C INDIA LIMITED|    1000000.0|  Paid|TL-11448| 12|    4|2019|
|   2|   2019-04-12|26/Apr/2019|A B C INDIA LIMITED|    1000000.0|  Paid|TL-11447| 12|    4|2019|
|   3|   2019-04-12|26/Apr/2019|A B C INDIA LIMITED|    1000000.0|  Paid|TL-11441| 12|    4|2019|
|   4|   2019-04-12|26/Apr/2019|A B C INDIA LIMITED|     100000.0|  Paid| OL-1113| 12|    4|2019|
|   5|   2019-04-12|26/Apr/2019|A B C INDIA LIMITED|     100000.0|  Paid| OL-1118| 12|    4|2019|
+----+-------------+-----------+-------------------+-------------+------+--------+---+-----+----+
only showing top 5 rows



In [42]:
topPurchaser = purchaser_df.groupBy('PurchaserName').agg(_sum('Denominations').alias('TotalAmount')).orderBy('TotalAmount', ascending=False)
topPurchaser.show(5, truncate=False)

+----------------------------------------------+-----------+
|PurchaserName                                 |TotalAmount|
+----------------------------------------------+-----------+
|FUTURE GAMING AND HOTEL SERVICES PR           |1.208E10   |
|MEGHA ENGINEERING AND INFRASTRUCTURES LI MITED|8.21E9     |
|QWIKSUPPLYCHAINPRIVATELIMITED                 |4.1E9      |
|HALDIA ENERGY LIMITED                         |3.77E9     |
|VEDANTA LIMITED                               |3.7565E9   |
+----------------------------------------------+-----------+
only showing top 5 rows



In [43]:
topPurchaser.orderBy('TotalAmount', ascending=False).show()

+--------------------+-----------+
|       PurchaserName|TotalAmount|
+--------------------+-----------+
|FUTURE GAMING AND...|   1.208E10|
|MEGHA ENGINEERING...|     8.21E9|
|QWIKSUPPLYCHAINPR...|      4.1E9|
|HALDIA ENERGY LIM...|     3.77E9|
|     VEDANTA LIMITED|   3.7565E9|
|ESSEL MINING AND ...|    2.245E9|
|WESTERN UP POWER ...|      2.2E9|
|KEVENTER FOODPARK...|     1.95E9|
|       MADANLAL LTD.|    1.855E9|
|BHARTI AIRTEL LIM...|     1.83E9|
|YASHODA SUPER SPE...|     1.62E9|
|UTKAL ALUMINA INT...|    1.353E9|
|DLF COMMERCIAL DE...|      1.3E9|
|MKJ ENTERPRISES L...|   1.2835E9|
|JINDAL STEEL AND ...|     1.23E9|
|B G SHIRKE CONSTR...|     1.17E9|
|DHARIWAL INFRASTR...|     1.15E9|
|CHENNAI GREEN WOO...|     1.05E9|
|BIRLACARBONINDIAP...|     1.05E9|
|   RUNGTA SONS P LTD|      1.0E9|
+--------------------+-----------+
only showing top 20 rows



In [44]:
n = 50
top_n_purchasers = topPurchaser.orderBy('TotalAmount', ascending=False).limit(n)

fig1 = px.bar(top_n_purchasers.toPandas(),
              y='PurchaserName',
              x='TotalAmount',
              color='TotalAmount',
              title=f'Top {n} Bonds Purchased Parties (Amount in Crores)',
              text_auto='.2s',
              orientation='h')

fig1.update_traces(textfont_size=12, textangle=0, cliponaxis=False, textposition='outside')
fig1.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', height=800)
fig1.show()

In [45]:
totalByYear = purchaser_df.groupBy('PurchaserName', 'Year').agg(_sum('Denominations').alias('TotalAmountByYear')).orderBy('PurchaserName', 'Year', 'TotalAmountByYear')
totalByYear.show(5, truncate=False)

+----------------------------------+----+-----------------+
|PurchaserName                     |Year|TotalAmountByYear|
+----------------------------------+----+-----------------+
|14 REELS PLUS LLP                 |2022|1.0E7            |
|A B C INDIA LIMITED               |2019|4000000.0        |
|AAKANKSHA BAHETY                  |2022|1000000.0        |
|AALAYA CONSTRUCTIONS              |2023|1.0E7            |
|AARISH SOLAR POWER PRIVATE LIMITED|2023|2.0E7            |
+----------------------------------+----+-----------------+
only showing top 5 rows



In [46]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

totalByYearPD = totalByYear.toPandas()
partyName = [row.PurchaserName for row in totalByYear.select('PurchaserName').distinct().collect()]


app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='Total Purchased per Year', style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='dropdown-selection',
        options=[{'label': party, 'value': party} for party in totalByYearPD['PurchaserName'].unique()],
        value='14 REELS PLUS LLP'
    ),
    dcc.Graph(id='graph-content')
])

@app.callback(
    Output('graph-content', 'figure'),
    [Input('dropdown-selection', 'value')]
)
def update_graph(value):
    df = totalByYearPD[totalByYearPD['PurchaserName'] == value]
    fig = px.bar(df, x='Year', y='TotalAmountByYear', color='TotalAmountByYear')
    fig.update_xaxes(type='category')
    fig.update_layout(bargap=0.3)

    return fig

if __name__ == '__main__':
     app.run_server(mode='inline', host="0.0.0.0", port=1005)


<IPython.core.display.Javascript object>